# PPR Dependence

I'm interested in how dramatically the value-based rankings of players change with the points per reception. My guess is that there will be large swings between 0, 0.5 and 1.0 points per reception. The implication would be that one-size fits all rankings are BS.

In [93]:
import os
import pandas as pd

data = pd.read_csv(os.path.join("..", "..", "data", "processed", "Projections-2019.csv"))

data.head()

,key,name,pos,team,bye,adp,rush_yds,kick_50,df_ints,fumbles,...,kick_0_19,df_tds,df_fumbles,rush_tds,pass_ints,df_sacks,kick_extra_points,kick_20_29,pass_tds,two_pts
0,barkley_RB_NYG,Saquon Barkley,RB,NYG,11.0,1.3,1267.0,NaN,NaN,4.0,...,NaN,NaN,NaN,9.6,0.0,NaN,NaN,NaN,NaN,0.6
1,elliott_RB_DAL,Ezekiel Elliott,RB,DAL,8.0,2.5,1364.0,NaN,NaN,2.4,...,NaN,NaN,NaN,8.6,0.0,NaN,NaN,NaN,NaN,NaN
2,mccaffrey_RB_CAR,Christian McCaffrey,RB,CAR,7.0,2.8,1055.6,NaN,NaN,1.8,...,NaN,NaN,NaN,6.6,0.0,NaN,NaN,NaN,NaN,0.0
3,kamara_RB_NO,Alvin Kamara,RB,NO,9.0,3.5,943.0,NaN,NaN,1.8,...,NaN,0.1,NaN,9.0,0.0,NaN,NaN,NaN,NaN,1.6
4,gordon_RB_LAC,Melvin Gordon,RB,LAC,12.0,6.0,1005.9,NaN,NaN,3.0,...,NaN,NaN,NaN,9.5,0.0,NaN,NaN,NaN,NaN,0.5


In [94]:
# create projections for each player

import numpy as np

points = {  # standard ESPN scoring
    "pass_yds": 0.04,
    "pass_tds": 4.0,
    "pass_ints": -2.0,
    "receptions": 0.0,  # the stat that matters here
    "reception_yds": 0.1,
    "reception_tds": 6.0,
    "rush_yds": 0.1,
    "rush_tds": 6.0,
    "fumbles": -2.0,
    "two_pts": 2.0,
    "kick_extra_points": 1.0,
    "kick_0_19": 3.0,
    "kick_20_29": 3.0,
    "kick_30_39": 3.0,
    "kick_40_49": 4.0,
    "kick_50": 5.0,
    "df_ints": 2.0,
    "df_tds": 6.0,
    "df_sacks": 1.0,
    "df_points_allowed_per_game": 0,
    "df_fumbles": 2.0,
    "df_safeties": 2.0,
}

data = data.fillna(0)

for ppr in [0.0, 0.5, 1.0]:
    points["receptions"] = ppr

    proj = "proj-" + str(ppr)  # col name
    data[proj] = data.apply(lambda x: sum(x[col] * pts for col, pts in points.items() if x[col]), axis=1)

data.head()

,key,name,pos,team,bye,adp,rush_yds,kick_50,df_ints,fumbles,...,rush_tds,pass_ints,df_sacks,kick_extra_points,kick_20_29,pass_tds,two_pts,proj-0.0,proj-0.5,proj-1.0
0,barkley_RB_NYG,Saquon Barkley,RB,NYG,11.0,1.3,1267.0,0.0,0.0,4.0,...,9.6,0.0,0.0,0.0,0.0,0.0,0.6,263.38,304.63,345.88
1,elliott_RB_DAL,Ezekiel Elliott,RB,DAL,8.0,2.5,1364.0,0.0,0.0,2.4,...,8.6,0.0,0.0,0.0,0.0,0.0,0.0,255.24,290.54,325.84
2,mccaffrey_RB_CAR,Christian McCaffrey,RB,CAR,7.0,2.8,1055.6,0.0,0.0,1.8,...,6.6,0.0,0.0,0.0,0.0,0.0,0.0,249.75,296.45,343.15
3,kamara_RB_NO,Alvin Kamara,RB,NO,9.0,3.5,943.0,0.0,0.0,1.8,...,9.0,0.0,0.0,0.0,0.0,0.0,1.6,247.98,292.18,336.38
4,gordon_RB_LAC,Melvin Gordon,RB,LAC,12.0,6.0,1005.9,0.0,0.0,3.0,...,9.5,0.0,0.0,0.0,0.0,0.0,0.5,219.14,245.69,272.24


In [95]:
# add value to each player

positions = ["QB", "RB", "WR", "TE", "DST", "K"]

data = data[data.adp > 0]

for pts in ["proj-0.0", "proj-0.5", "proj-1.0"]:
    # map from each position to the numberof players in that position
    # drafted during the first 100 picks of a draft
    # that's the index of the replacement player
    # see: https://www.footballguys.com/05vbdrevisited.htm
    replacement = {}
    for pos in positions:
        replacement[pos] = len(data[(data.pos == pos) & (data.adp <= 100.0)])

    replace_pts = {}
    for pos, ind in replacement.items():
        replace_pts[pos] = data[data.pos == pos][ind:ind+1][pts].values[0]
                                                              
    value = pts + "-value"
    data[value] = data.apply(lambda x: x[pts] - replace_pts[x["pos"]], axis=1)
    
data.head()

,key,name,pos,team,bye,adp,rush_yds,kick_50,df_ints,fumbles,...,kick_extra_points,kick_20_29,pass_tds,two_pts,proj-0.0,proj-0.5,proj-1.0,proj-0.0-value,proj-0.5-value,proj-1.0-value
0,barkley_RB_NYG,Saquon Barkley,RB,NYG,11.0,1.3,1267.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.6,263.38,304.63,345.88,166.09,187.64,209.19
1,elliott_RB_DAL,Ezekiel Elliott,RB,DAL,8.0,2.5,1364.0,0.0,0.0,2.4,...,0.0,0.0,0.0,0.0,255.24,290.54,325.84,157.95,173.55,189.15
2,mccaffrey_RB_CAR,Christian McCaffrey,RB,CAR,7.0,2.8,1055.6,0.0,0.0,1.8,...,0.0,0.0,0.0,0.0,249.75,296.45,343.15,152.46,179.46,206.46
3,kamara_RB_NO,Alvin Kamara,RB,NO,9.0,3.5,943.0,0.0,0.0,1.8,...,0.0,0.0,0.0,1.6,247.98,292.18,336.38,150.69,175.19,199.69
4,gordon_RB_LAC,Melvin Gordon,RB,LAC,12.0,6.0,1005.9,0.0,0.0,3.0,...,0.0,0.0,0.0,0.5,219.14,245.69,272.24,121.85,128.70,135.55


In [115]:
# add ranks, sort and print top 50 players

from sklearn.metrics import r2_score

ranks = ["0.0-rank", "0.5-rank", "1.0-rank"]

for i, pts in enumerate(["proj-0.0-value", "proj-0.5-value", "proj-1.0-value"]):
    data = data.sort_values(pts, ascending=False)
    data[ranks[i]] = np.arange(len(data)) + 1
    
data["diff"] = data[ranks[2]] - data[ranks[0]]
data["abs_diff"] = abs(data["diff"])
data = data.sort_values(ranks[0], ascending=True)

top_100 = data[:101]

for r in ranks:
    print(r, r2_score(top_100[r], top_100["adp"]))
    
large_diff = top_100.sort_values("abs_diff", ascending=False)[:10]


cols = ["name", "pos", "adp"] + ranks + ["diff", "abs_diff"]
large_diff[cols]
    
# data[cols].head(n=50)


0.0-rank 0.47465719277810137
0.5-rank 0.4739491035155383
1.0-rank 0.43610480669845253


,name,pos,adp,0.0-rank,0.5-rank,1.0-rank,diff,abs_diff
76,Rashaad Penny,RB,80.8,88,126,164,76,76
82,Jordan Howard,RB,85.8,71,87,125,54,54
80,Darrell Henderson,RB,84.3,87,94,126,39,39
55,Tyreek Hill,WR,56.5,80,86,113,33,33
38,Sony Michel,RB,40.0,19,32,42,23,23
68,Tevin Coleman,RB,70.3,64,76,87,23,23
163,Jordan Reed,TE,164.3,83,68,63,-20,20
32,Derrick Henry,RB,33.0,10,18,30,20,20
11,Michael Thomas,WR,12.5,31,19,15,-16,16
19,Nick Chubb,RB,19.8,16,25,32,16,16
